<a href="https://colab.research.google.com/github/emerenan/validador/blob/master/validador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import numpy as np
be = ['COD_INT_PROD','MAT_PRINCIP','MAT_SECUND','DT_NASC','TIPO_BENEF','SEXO','DT_ADESAO','DT_EXCLUSAO','CPT','DT_FINAL_CPT','COD_EMPRESA','TIPO_PATROCIN','HIST_BENEF']
co = ['COD_INT_PROD', 'MAT_PRINCIP', 'MAT_SECUND', 'ANO_MES_COMP', 'CONTRAPRESTACAO']

def readfile(path, colname, ops):
    """ Método para leitura de arquivo no formato .csv
    
    Arg: \n
        path: caminho onde o arquivo está localizado. \n
        colname: lista com o nome das colunas a serem atribuidas ao dataframe. \n
        ops: nome da operadora a ser inserido no Dataframe
    Returns: \n
        Retorna um dataframe com as informações do 
    """
    try:
        df = pd.read_csv(path, sep=';', names=colname, error_bad_lines=False, warn_bad_lines=True, encoding='U8')
        # print(df.head())
        # print('-----------------------------------------------------------------------')
        # print(df.isnull().sum())
        # print('-----------------------------------------------------------------------')
        # print(df.info())
        # print('-----------------------------------------------------------------------')
        df['OPERADORA'] = ops
        df = df.fillna(0)
        
        return df
    
    except pd.io.common.ParserError:
        print("Dados não lidos")

def change_type(df, col, tipo):
    """
    Método para alterar o tipo da coluna para poder cruzar com o DB.

    Args: \n
        df: Dataframe que terá a coluna passada modificada. \n
        col: Lista com a(s) coluna(s) que será(ão) modificada(s). \n
        tipo: O tipo do dado que a coluna irá se tornar. \n
    Returns: \n
        Retorna um dataframe com uma coluna modificada do tipo passado no argumento.
    """
    for i in col:
        df[i] = df[i].astype(tipo)
    return df

def cont_repet(df, coluna):
    """ Método para contagem de valores na coluna, mostrando ou não os valores repetidos.
    Arg: \n
        df: nome da dataframe que será usado para contagem. \n
        coluna: nome da coluna que deverá ser feita a contagem dos valores. \n

    Returns: \n
        Retorna um dicionario com a contagem do valores, assim como os repetidos.\n

    """
    # dicionário para salvar os valores contados
    index_cont = {}

    # Extraindo os valores da coluna e salvando na variável
    col = df[coluna]

    # Iterando entre a coluna do dataframe
    for index in col:
        # Se o index for existente, add 1
        if index in index_cont.keys():
            index_cont[index] += 1
        else:
            index_cont[index] = 1
    return index_cont
    # return index_cont
    # Dicionario para contar os valores sem repetição e com repetição
    rep = {'sem_repeticao' : 0, 'com_repeticao' : 0}

    # Iterando sobre o dicionario index_col e contando os valores sem repetição e com repetição
    for key, value in index_cont.items():
        if value == 1:
            rep['sem_repeticao'] += 1
        else:
            rep['com_repeticao'] += 1
    
    # return rep

def real_float(df, col):
    """ Método para modificar as colunas do tipo money para float e salvar o arquivo num caminho.
    
    Args: \n
        df: Dataframe que terá a coluna passada modificada. \n
        col: Lista com a(s) coluna(s) que será(ão) modificada(s). \n
        path: Caminho onde será salvo o novo dataframe com a coluna modificada.
    Returns: \n
        Retorna um dataframe com uma coluna modificada do tipo money para float.
    """
    for i in col:
        df[i] = [x.replace('R$ ', '') for x in df[i]]
        df[i] = [x.replace('.', '') for x in df[i]]
        df[i] = [x.replace(',', '.') for x in df[i]]
        df[i] = df[i].astype(float)
    return df  
    # df.to_csv(path, index=False)

print('Abaixo temos as opções de arquivos a serem validados. ')
print('----------------------------------------------------')
print('be: Para arquivo referente a Tabela Beneficiarios.\n' + \
      'co: Para arquivo referente a Tabela contraprestação. \n' + \
      'si: Para arquivo referente a Tabela de Sinistro. \n' + \
      'pl: Para arquivo referente a Tabela de Planos. \n' + \
      'pr: Para arquivo referente a Tabela de Prestador. \n' + \
      'pc: Para arquivo referente a Tabela Plano Cidades. \n' + \
      'pp: Para arquivo referente a Tabela Propria. \n')
print('----------------------------------------------------')

tp_file = input('Tipo do arquivo. \n')

# Quando o tp_file for be(Tabela de beneficarios.)
while tp_file == 'be':
    caminho = str(input('Digite o caminho do arquivo. \n'))
    ops = int(input('Entre com o cod. da operadora. \n'))

    # Lendo o arquivo passado
    data_be = readfile(caminho, be, ops)
    
    colunas = ['DT_EXCLUSAO', 'DT_FINAL_CPT']
    colstr = ['COD_INT_PROD', 'MAT_PRINCIP', 'MAT_SECUND', 'DT_NASC', 'DT_ADESAO', 'DT_EXCLUSAO', 'DT_FINAL_CPT', 'COD_EMPRESA']
    data_be = change_type(data_be, colunas, 'int')
    data_be = change_type(data_be, colstr, str)
    
    # Criando a Coluna de Situação do Beneficiario
    data_be['SITUACAO'] = 'ativo'

    # Condificonal para definir o situação do Beneficario (ativo ou inativo)
    for index, row in data_be.iterrows():
        if row['DT_EXCLUSAO'] > row['DT_ADESAO']:
            data_be.loc[index, 'SITUACAO'] = 'inativo'
        data_be['SITUACAO'] = data_be['SITUACAO'].astype(str)
    
    output = '''
                        Totalizador de Beneficiários                    
    -------------------------------------------------------------------------
    Total de Beneficarios: {beneficiarios}
    Valores Duplicados: {duplicatas}
    Ativos: {ativos}
    Inativos: {inativos}
    '''
    # Contagem de Beneficarios:
    benef =  str(data_be.MAT_PRINCIP.count())
    # Contagem de linhas duplicadas
    duplicatas = len(data_be)- len(data_be.drop_duplicates())
    # Contagem de situação dos clientes
    sit = cont_repet(data_be, 'SITUACAO')
    for key, value in sit.items():
        if key == 'ativo':
            at = value
        else:
            inat = value
    

    print(output.format(beneficiarios=benef, duplicatas=duplicatas, ativos=at, inativos=inat))
    break
else:
    print('Nenhum valor passado!')

#data_be = readfile('/content/419851_BE_082020.csv', be, 1735)
# data_co = readfile('/content/310522_CO_012020_072020.csv')
# data_si = readfile('/content/310522_SI_012020_062020.csv')
# data_pl = readfile('/content/310522_PL_08052020.csv')
# data_pr = readfile('/content/310522_PR_052020.csv')
# data_pp = readfile('/content/310522_PP_201912.csv')
# data_pc = readfile('/content/310522_PC_08052020.csv')  

Abaixo temos as opções de arquivos a serem validados. 
----------------------------------------------------
be: Para arquivo referente a Tabela Beneficiarios.
co: Para arquivo referente a Tabela contraprestação. 
si: Para arquivo referente a Tabela de Sinistro. 
pl: Para arquivo referente a Tabela de Planos. 
pr: Para arquivo referente a Tabela de Prestador. 
pc: Para arquivo referente a Tabela Plano Cidades. 
pp: Para arquivo referente a Tabela Propria. 

----------------------------------------------------
Tipo do arquivo. 
be
Digite o caminho do arquivo. 
/content/419851_BE_082020.csv
Entre com o cod. da operadora. 
1735

                        Totalizador de Beneficiários                    
    -------------------------------------------------------------------------
    Total de Beneficarios: 13538
    Valores Duplicados: 0
    Ativos: 8054
    Inativos: 5484
    


In [ ]:
pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 8.5MB/s 


In [86]:
qtd_tabelas = int(input("Quantas tabelas serão analisadas? \n"))
tabelas = []

if qtd_tabelas == 1:
    uma = input('Qual o nome da tabela? \n')
    tabelas.append(uma)
elif qtd_tabelas == 2:
    um, dois = input('Qual o nome das Tabelas: ').split(',')
    tabelas.append(um)
    tabelas.append(dois)
else:
    print('Valor acima da quantidades de tabelas do Layout CTS!')
print(tabelas)

"""

for tabela in tabelas:
    if tabela == 'be':
        print('validando a tabela be')
    if tabela == 'co':
        print('validando a tabela co')
    if tabela == 'si':
        print('validando a tabela si')
    if tabela == 'pl':
        print('validando a tabela pl')
    if tabela == 'pr':
        print('validando a tabela pr')
    if tabela == 'pp':
        print('validando a tabela pp')
    if tabela == 'pc':
        print('validando a tabela pc')
"""






Quantas tabelas serão analisadas? 
2
Qual o nome das Tabelas: be, co
['be', ' co']


"\n\nfor tabela in tabelas:\n    if tabela == 'be':\n        print('validando a tabela be')\n    if tabela == 'co':\n        print('validando a tabela co')\n    if tabela == 'si':\n        print('validando a tabela si')\n    if tabela == 'pl':\n        print('validando a tabela pl')\n    if tabela == 'pr':\n        print('validando a tabela pr')\n    if tabela == 'pp':\n        print('validando a tabela pp')\n    if tabela == 'pc':\n        print('validando a tabela pc')\n"